### Libraries

In [22]:
from typing import List
from polygon import RESTClient      # https://github.com/polygon-io/client-python
import os
import requests
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.tools import tool
from tavily import TavilyClient
import pandas as pd
from bs4 import BeautifulSoup
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
import keyring
import time

In [3]:
tavily_client = TavilyClient(api_key=keyring.get_password('openai', 'key_for_windows'))
polygon_client = RESTClient(api_key=keyring.get_password('polygon', 'key_for_windows'))
OPENAI_API_KEY = keyring.get_password('openai', 'key_for_windows')
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0, api_key=OPENAI_API_KEY)

### SEC 파일링을 통해 비재무 지표 요약하기

https://www.sec.gov/search-filings/edgar-application-programming-interfaces

In [ ]:
headers = {'User-Agent': 'ahn283@gmail.com'}
# get tickers
company_ticker_url = "https://www.sec.gov/files/company_tickers.json"
response = requests.get(company_ticker_url, headers=headers)
company_tickers = response.json()
company_data = pd.DataFrame.from_dict(company_tickers, orient='index')
# create cik column
company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)
ticker = 'NVDA'.upper()
company_data.set_index('ticker', inplace=True)

cik = company_data.loc['NVDA', 'cik_str']

base_url = f'https://data.sec.gov/submissions/CIK{cik}.json'
response = requests.get(base_url, headers=headers)

data = response.json()
fillings = data.get('filings', {}).get('recent', {})
forms = fillings.get('form', [])
dates = fillings.get('filingDate', [])
accession_numbers = fillings.get('accessionNumber', [])
document_urls = fillings.get('primaryDocument', [])
df = pd.DataFrame({
    'form': forms,
    'date': dates,
    'accession_number': accession_numbers,
    'document_url': document_urls
})
df_filtered = df[df['form'].isin(['10-K', '10-Q', '8-K'])]
latest_filings = df_filtered.sort_values('date', ascending=False).drop_duplicates('form')

In [16]:
results = {}
for _, row in latest_filings.iterrows():
    form_type = row['form']
    date = row['date']
    accession_number = row['accession_number']
    document_url = row['document_url']
    filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_number.replace('-', '')}/{document_url}"
    response = requests.get(filing_url, headers=headers)
    if response.status_code != 200:
        results[form_type] = f"Failed to retrieve the filing form {filing_url}"
    else:
        soup = BeautifulSoup(response.content, 'html.parser')
        text_content = soup.get_text(separator='\n')
        results[form_type] = {
            'date': date,
            'url': filing_url,
            'content': text_content
        }

In [17]:
results

{'10-Q': {'date': '2024-11-20',
  'url': 'https://www.sec.gov/Archives/edgar/data/1045810/000104581024000316/nvda-20241027.htm',
  'content': '\n\n\n\n\n\n\n\n\n\nnvda-20241027\n0001045810\n1/26\n2025\nQ3\nFALSE\nhttp://fasb.org/us-gaap/2024#AccruedLiabilitiesCurrent\nhttp://fasb.org/us-gaap/2024#AccruedLiabilitiesCurrent\n0\n550\n456\nxbrli:shares\niso4217:USD\niso4217:USD\nxbrli:shares\nxbrli:pure\nnvda:segment\n0001045810\n2024-01-29\n2024-10-27\n0001045810\n2024-11-15\n0001045810\n2024-07-29\n2024-10-27\n0001045810\n2023-07-31\n2023-10-29\n0001045810\n2023-01-30\n2023-10-29\n0001045810\n2024-10-27\n0001045810\n2024-01-28\n0001045810\nus-gaap:CommonStockMember\n2024-07-28\n0001045810\nus-gaap:AdditionalPaidInCapitalMember\n2024-07-28\n0001045810\nus-gaap:AccumulatedOtherComprehensiveIncomeMember\n2024-07-28\n0001045810\nus-gaap:RetainedEarningsMember\n2024-07-28\n0001045810\n2024-07-28\n0001045810\nus-gaap:RetainedEarningsMember\n2024-07-29\n2024-10-27\n0001045810\nus-gaap:Accumulat

In [26]:
@tool
def get_latest_filing_content(ticker: str) -> dict: 
    """get recent 10-K, 10-Q, 8-K filing and extract filing contents for a given ticker"""
    headers = {'User-Agent': 'ahn283@gmail.com'}
    # get company tickers and cik_str
    company_ticker_url = 'https://www.sec.gov/files/company_tickers.json'
    response = requests.get(company_ticker_url, headers=headers)
    if response.status_code != 200:
        return f"Failed to retrieve company tickers."
    company_tickers = response.json()
    company_data = pd.DataFrame.from_dict(company_tickers, orient='index')
    # create cik_str column for the CIK
    company_data['cik_str'] = company_data['cik_str'].astype(str).str.zfill(10)
    ticker = ticker.upper()
    company_data.set_index('ticker', inplace=True)
    if ticker not in company_data.index:
        return f"Ticker {ticker} not found"
    
    cik = company_data.loc[ticker, 'cik_str']
    base_url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    response = requests.get(base_url, headers=headers)
    if response.status_code != 200:
        return f"Failed to retrieve filings for CIK {cik}"
    data = response.json()
    filings = data.get('filings', {}).get('recent', {})
    forms = fillings.get('form', [])
    dates = fillings.get('filingDate', [])
    accession_numbers = filings.get('accessionNumber', [])
    document_urls = filings.get('primaryDocument', [])
    df = pd.DataFrame({
        'form': forms,
        'date': dates,
        'accession_number': accession_numbers,
        'document_url': document_urls
    })
    df_filtered = df[df['form'].isin(['10-K', '10-Q', '8-K'])]
    latest_filings = df_filtered.sort_values('date', ascending=False).drop_duplicates('form')
    results = {}
    for _, row in latest_filings.iterrows():
        form_type = row['form']
        date = row['date']
        accession_number = row['accession_number']
        document_url = row['document_url']
        filing_url = f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_number.replace('-', '')}/{document_url}"
        response = requests.get(filing_url, headers=headers)
        if response.status_code != 200:
            results[form_type] = f"Failed to retrieve the filing form {filing_url}"
        else:
            soup = BeautifulSoup(response.content, 'html.parser')
            text_content = soup.get_text(separator='\n')
            results[form_type] = {
                'date': date,
                'url': filing_url,
                'content': text_content
            }
    
    # LCEL chain
    def summarize_filings(filings: dict) -> dict: 
        prompt = ChatPromptTemplate.from_template(
            """다음은 {form_type} 파일링의 내용입니다. 주요 재무 지표, 중요한 사실들, 
            그리고 구체적인 세부 사항을 포함하여 요약해주세요.
            최대한 풍부한 요약이 되게끔 해주세요.
            
            각 보고사 유형에 따라 다음과 같은 중요 정보들을 포함해야 합니다:
            
            10-K (연간 보고서):
            - 주요 재무 지표 (정확한 수치와 함께 매출, 순이익, EPS 등)
            - 사업 개표 및 주요 제품/서비스 (구체적인 제품며이나 서비스명 포함)
            - 주요 시장 및 고객 (가능한 경우 주요 고객사 이름 포함)
            - 경영진의 주요 변동 사항 (해당되는 경우 구체적인 이름과 직책 포함)
            - 중요한 위험 요인 (구체적인 예시와 함께)
            - 향후 전략 및 전망
            - 주요 소송 또는 규제 이슈 (구체적인 사건명이나 관련 기관명 포함)
            
            10-Q (분기 보고서):
            - 분기별 주요 재무 지표 (정확한 수치와 전년 동기 대비 변동률)
            - 주요 제품/서비스의 실적 (구체적인 제품며이나 서비스명과 함께)
            - 시장 동향 및 경쟁 상황 (가능한 경우 경쟁사 이름 포함)
            - 단기적인 위험 요소나 기회 (구체적인 예시와 함께)
            - 주요 운영 변경 사항 (해당되는 경우 구체적인 내용 포함)
            
            8-K (수시 보고서):
            - 보고 이벤트의 성격 (예: 경영진 변경, 인수합병, 중요 계약 체결 등)
            - 해당 이벤트의 주요 내용 (관련된 모든 당사자의 이름, 금액, 날짜 등 포함)
            - 회사에 미치는 잠재적 영향 (가능한 경우 구체적인 수치 예측 포함)
            - 관련된 중요 인물의 배경 (해당되는 경우)
            
            각 항목에 대해 가능한 한 구체적인 세부 사항 (이름, 숫자, 날짜 등)을 포함해주세요.
            그러나 전체 요약은 간결해야 하며, 각 항목은 1-3문장으로 제한해주세요.
            
            파일링 내용:
            {text}
            
            요약:"""
        )
        chain = prompt | llm | StrOutputParser()
        
        # preparing inputs for processing batches
        inputs = [
            {"text": filing_data['content'], 'form_type': form_type}
            for form_type, filing_data in filings.items()
            if isinstance(filing_data, dict) and 'content' in filing_data
        ]
        
        summaries = chain.batch(inputs)
        
        return {
            form_type: summary
            for (form_type, filing_data), summary in zip(filings.items(), summaries)
            if isinstance(filing_data, dict) and 'content' in filing_data
        }
        
    # collect filing and summarize
    summaries = summarize_filings(results)
    
    return summaries

In [27]:
get_latest_filing_content({'ticker': 'NVDA'})

{'10-Q': '### NVIDIA Corporation 10-Q Summary for the Quarter Ended October 27, 2024\n\n#### 주요 재무 지표\n- **매출**: $35,082 million (전년 동기 대비 94% 증가, 전 분기 대비 17% 증가)\n- **순이익**: $19,309 million (전년 동기 대비 109% 증가)\n- **EPS (희석 기준)**: $0.78 (전년 동기 대비 111% 증가)\n- **총 자산**: $96,013 million (2024년 1월 28일 대비 증가)\n- **현금 및 현금성 자산**: $9,107 million\n\n#### 사업 개요 및 주요 제품/서비스\nNVIDIA는 데이터 센터, 게임, 전문 시각화 및 자동차 분야에서 GPU 및 AI 솔루션을 제공하는 선도적인 기업입니다. 주요 제품으로는 Hopper 아키텍처 기반의 데이터 센터 시스템과 GeForce RTX 40 시리즈 GPU가 있습니다.\n\n#### 주요 시장 및 고객\n- **데이터 센터**: 매출의 약 50%가 클라우드 서비스 제공업체에서 발생.\n- **게임**: GeForce RTX 40 시리즈 GPU의 판매 증가.\n- **전문 시각화**: RTX GPU 워크스테이션의 지속적인 성장.\n\n#### 경영진의 주요 변동 사항\n- Aarti Shah (이사)와 Ajay K. Puri (전 세계 영업 부사장)가 각각 주식 매각을 위한 Rule 10b5-1 거래 계획을 채택.\n\n#### 중요한 위험 요인\n- **공급망 불확실성**: 반도체 공급망의 불확실성과 고객 수요 예측의 어려움.\n- **수출 통제**: 미국 정부의 수출 통제가 중국 및 기타 지역으로의 제품 판매에 영향을 미칠 수 있음.\n- **법적 소송**: 증권 관련 소송이 진행 중이며, 결과에 따라 재무적 영향을 받을 수 있음.\n\n#### 향후 전략 및 전망\nNVIDIA는 데이터 센터 및 AI 솔루션에 대한 수요 증가에 대응하기 위

### 경쟁사 관련 증권 뉴스 가져오기

In [34]:
# get news about competitors
@tool
def collect_competitor_news(ticker, news_count, interval=10):
    """Get recent news about a company of a given ticker
    
    Args:
        ticker (str): The company ticker
        news_count (int) : number how many news will we collect. Basic Number is 10.
    Returns:
        Dict(List): A Dict of competitor's news, each containing recent news articles's description
        
    Example:
        response = competitor_news("ticker": "AAPL", "news_count":10)
        response = {"MSFT: ["US stock....", "MS invests on...", ...]}
    """
    related_companies = polygon_client.get_related_companies(ticker)
    competitors = [i.ticker for i in related_companies]
    
    time.sleep(interval)
    
    competitor_news = {}
    for c in competitors:
        api_key = keyring.get_password('polygon', 'key_for_windows')
        api_url = f'https://api.polygon.io/v2/reference/news?ticker={c}&order=desc&limit={news_count}&sort=published_utc&apiKey={api_key}'
        result = requests.get(api_url).json()
        try:
            competitor_news[c] = [i['description'] for i in result['results']]
            print(f"completed: {c}")
        except Exception as e:
            print(f'Error: {c}\n{e}')
            continue
        time.sleep(interval)
    
    return competitor_news

In [35]:
collect_competitor_news({'ticker':'NVDA', 'news_count':3, 'interval': 10})

comleted : AMD
comleted : META
comleted : GOOGL
comleted : TSLA
comleted : GOOG
comleted : MSFT
comleted : AMZN
'results'
comleted : INTC
comleted : AVGO


{'AMD': ['The article examines the historical performance of AI and blockchain-related stocks, as well as broader market indexes, to assess the potential for AI stocks to continue dominating in 2025. It suggests that while the markets are likely to rise, not all megatrends or the companies involved with them are created equal, and investors should focus on established players or passive index funds rather than speculating on individual stocks.',
  "The article recommends three tech stocks to consider as investors rebalance their portfolios for 2025: AMD, IBM, and Universal Display. AMD is seen as a solid AI hardware investment, IBM's AI services are expected to see soaring demand, and Universal Display's OLED technology is finding more real-world use cases.",
  "The article discusses the potential challenges Nvidia may face in the AI market, despite its recent success. It suggests that the rise of AI could lead to a $15.7 trillion addressable market, but history shows that new technolo

In [36]:
@tool
def collect_company_news(company_name: str) -> str:
    """Collect recent news for the given company"""
    search_result = tavily_client.search(query=f"recent news about {company_name}", days=7)
    return f"Collected news and market data for {company_name}: \n{search_result}"

@tool
def collect_market_news(sector: str) -> str:
    """Collect recent market data for the given company's sector."""
    search_results = tavily_client.search(query=f"{sector} industry news", days=7)
    return f"Collected news and market data for {sector}: {search_results}"

@tool
def scrape_webpages(urls: List[str]) -> str:
    """Scrape the provided web pages for detailed information."""
    loader = WebBaseLoader(urls)
    docs = loader.load()
    return '\n\n'.join(
        [f'<Document name="{doc.metadata.get("title", "")}">\n(doc.page_content)\n</Document>'
         for doc in docs]
    )